# SUN database image categories

- The following image folders are all indoors
- Some indoor folders were not included since they had too many images with poor image characteristics 
- Stimuli Selection Guide: https://docs.google.com/document/d/1uwWAMf9fNJgzQjiYZTVynn7WfSLvxdZzT7V5uWz-nuU/edit#heading=h.l11npecuj44t

In [89]:
folders = ["a/archive", "a/art_gallery","a/attic", "a/auditorium", "b/ballroom", "b/banquet_hall", "b/bar", "b/bathroom"
           ,"b/bedroom", "b/bistro/indoor","b/bookstore", "c/cafeteria","c/childs_room","c/classroom","c/closet"
           ,"c/coffee_shop","c/computer_room","c/conference_room","c/courtroom","d/dining_room","d/dorm_room","g/game_room"
           ,"g/gymnasium/indoor","h/home_office","h/hotel_room","j/jewelry_shop","k/kindergarden_classroom","k/kitchen"
           ,"k/kitchenette","l/laundromat","l/lecture_room","l/library/indoor","l/living_room","l/locker_room","m/music_store"
           ,"n/nursery","o/office","p/pantry","p/pharmacy","p/playroom","r/reception","r/recreation_room","r/restaurant"
           ,"s/shoe_shop","s/shopping_mall/indoor","s/staircase","s/supermarket","s/sushi_bar","t/toyshop","u/utility_room"
           ,"w/waiting_room","w/wet_bar","w/wine_cellar/bottle_storage","y/youth_hostel"]

print(len(folders))

54


In [13]:
import os 
from PIL import Image 
import operator

def imageSize(rootdir, folders):
    size_dict = {}
    for folder in folders:
        folderpath = os.path.join(rootdir, folder)
        for image in os.listdir(folderpath):
            imagepath = os.path.join(rootdir, folder, image)
            PIL_image = Image.open(imagepath)
            width, height = PIL_image.size 
            imageSize = tuple([width, height])
            if imageSize not in size_dict:
                size_dict[imageSize] = 1
            else:
                size_dict[imageSize] += 1
    
    return size_dict
        
    
    
path = "/Users/prachi/Documents/SUN397"
SUN_size = imageSize(path, folders)   

max(SUN_size.items(), key=operator.itemgetter(1))[0]

SUN_size[(256, 256)]



2793

In [1]:
def imageSizeCategoryDistribution(rootdir, folders, width, height):
    """
    Find out how many images in each folder are greater than are equal to dimensions passed through
    """
    count_folders = []
    for folder in folders:
        count = 0
        folderpath = os.path.join(rootdir, folder)
        for image in os.listdir(folderpath):
            imagepath = os.path.join(rootdir, folder, image)
            PIL_image = Image.open(imagepath)
            w, h = PIL_image.size 
            if w >= width:
                if h >= height:
                    count+= 1
        count_folders.append(count)
    
    return count_folders
        
path = "/Users/prachi/Documents/SUN397"
        
imageSizeCategoryDistribution(path, folders, 256, 256)

# Conclusion: Too many folders have NO images that are (256, 256) 

In [78]:
p = '/Users/prachi/Desktop/sun_afwmvcfqgvxaiubd.jpg'


def resizeImage(imagepath):
    split_path = imagepath.split('/')
    file = split_path[-1]
    split_file = file.split('.')
    image = split_file[0]
    
    foo = Image.open(imagepath)
    foo = foo.resize((730,500),Image.ANTIALIAS)
    foo.save("/Users/prachi/Desktop/" + image + "_scaled.jpg",optimize=True,quality=95)
     # The saved downsized image size is 22.9kb
        
resizeImage(p)

Testing Resizing images:
- Scale up
- Scale down 

### Check how many SUN images have an Aspect Ratio (width : height) of 730 : 530 

In [87]:
from PIL import Image 
import os

def getAspectRatio(imagepath):
    """
    Returns the aspect ratio (width/height) of the image 
    """
    try:
        im = Image.open(imagepath)
    except ValueError as err:
        print(err)
        print("Error on image: ", img)
        
    width, height = im.size
    
#     aspect_ratio = round(width/height, 1) # rounds to the nearest tenth
    aspect_ratio = width/height

    
    return aspect_ratio, width, height 


def AspectRatioDistribution(rootdir, folders):
    """
    Check how many images fit within a certain criterion for aspect ratio
    """
    count = 0
    count_not = 0
    count_height_530 = 0
    for folder in folders:
        folderpath = os.path.join(rootdir, folder)
        for image in os.listdir(folderpath):
            imagepath = os.path.join(rootdir, folder, image)
            aspect_ratio, width, height = getAspectRatio(imagepath)
            if (730/530) <= aspect_ratio <= 1.5:
                count += 1
            else:
                count_not += 1
            if height == 530:
                count_height_530 +=1 
#                 print(imagepath)
    return count, count_not

path = "/Users/prachi/Documents/MeanDepth_SUN"

AspectRatioDistribution(path, folders)

(3061, 18871)

In [94]:
def filterbyAspectRatio(rootdir, folders):
    """
    Delete images that have an aspect ratio of of range: 
    1.37-1.6 inclusive 
    
    """
    paths_keep = []
    for folder in folders:
        folderpath = os.path.join(rootdir, folder)
        for image in os.listdir(folderpath):
            imagepath = os.path.join(rootdir, folder, image)
            aspect_ratio, width, height = getAspectRatio(imagepath)
            if (730/530) <= aspect_ratio <= 1.6:
                paths_keep.append(imagepath)
            else:
                os.remove(imagepath)
    
    return paths_keep, len(paths_keep)

In [95]:
path = "/Users/prachi/Documents/MeanDepth_SUN"

paths_filtered, imagecount = filterbyAspectRatio(path, folders)

In [104]:
def resizeImage(imagepath):
    """
    Scale just the height dimension to 530, and then use that proportional change in height to scale the width dimension accordingly. 
    This way the aspect ratio from the original image to the scaled image is the same. 
    """
    try:
        im = Image.open(imagepath)
    except ValueError as err:
        print(err)
        print("Error on image: ", img)
        
    width, height = im.size
    aspect_ratio = width/height
    
    # the new width is calculated such that the aspect ratio does not change 
    # new height is set to 530, so the width is proportionally increased the 
      # same amount the height was 
    new_width = int(aspect_ratio * 530)

    foo = Image.open(imagepath)
    foo = foo.resize((new_width,530),Image.ANTIALIAS)
    foo.save(imagepath ,optimize=True,quality=95)
     # The saved downsized image size is 22.9kb

resizeImage(p)


def resizeDatabase(rootdir, folders):
    """
    Run resizeImage on all images
    """
    for folder in folders:
        folderpath = os.path.join(rootdir, folder)
        for image in os.listdir(folderpath):
            imagepath = os.path.join(rootdir, folder, image)
            try:
                resizeImage(imagepath)
            except:
                print("Failed at: ", imagepath)
    
path = '/Users/prachi/Documents/MeanDepth_SUN'  

resizeDatabase(path, folders)

Failed at:  /Users/prachi/Documents/MeanDepth_SUN/c/closet/sun_absahzamlrylkxyn.jpg
Failed at:  /Users/prachi/Documents/MeanDepth_SUN/c/conference_room/sun_aesylgnpqdbvuxoq.jpg
Failed at:  /Users/prachi/Documents/MeanDepth_SUN/g/game_room/sun_aopfaqlllpvzhrak.jpg
Failed at:  /Users/prachi/Documents/MeanDepth_SUN/g/gymnasium/indoor/sun_agaaettuwibuogtc.jpg
Failed at:  /Users/prachi/Documents/MeanDepth_SUN/g/gymnasium/indoor/sun_aiivklwtzhqcwvfo.jpg
Failed at:  /Users/prachi/Documents/MeanDepth_SUN/s/shopping_mall/indoor/sun_avwzjsijaxnwuzjx.jpg
Failed at:  /Users/prachi/Documents/MeanDepth_SUN/w/waiting_room/sun_aicytusmthfvqcwc.jpg
Failed at:  /Users/prachi/Documents/MeanDepth_SUN/y/youth_hostel/.DS_Store
